In [1]:
!pip install datasets transformers[sentencepiece]

     |████████████████████████████████| 312 kB 5.1 MB/s 
     |████████████████████████████████| 3.8 MB 46.1 MB/s 
     |████████████████████████████████| 212 kB 65.5 MB/s 
     |████████████████████████████████| 134 kB 17.3 MB/s 
     |████████████████████████████████| 67 kB 4.7 MB/s 
     |████████████████████████████████| 1.1 MB 50.6 MB/s 
     |████████████████████████████████| 127 kB 56.7 MB/s 
     |████████████████████████████████| 271 kB 40.6 MB/s 
     |████████████████████████████████| 94 kB 2.6 MB/s 
     |████████████████████████████████| 144 kB 56.5 MB/s 
     |████████████████████████████████| 895 kB 52.7 MB/s 
     |████████████████████████████████| 6.5 MB 34.6 MB/s 
     |████████████████████████████████| 596 kB 40.0 MB/s 
     |████████████████████████████████| 1.2 MB 47.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
   

In [2]:
from datasets import load_dataset

# This can take a few minutes to load, so grab a coffee or tea while you wait!
raw_datasets = load_dataset("code_search_net", "java")

Downloading:   0%|          | 0.00/2.60k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/1.06G [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset code_search_net downloaded and prepared to /root/.cache/huggingface/datasets/code_search_net/java/1.0.0/80a244ab541c6b2125350b764dc5c2b715f65f00de7a56107a28915fac173a27. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
raw_datasets["train"]

Dataset({
    features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'],
    num_rows: 454451
})

In [4]:
print(raw_datasets["train"][123456]["whole_func_string"])

public void load(final boolean recursive, final boolean replace, final boolean inNewThread)
    {
        if (inNewThread)
        {
            new Thread()
            {
                public void run()
                {
                    load(recursive, replace, false);
                }
            }.start();
            return;
        }
        if (!populationInProgress)
        {
            synchronized (this.populationLock)
            {
                this.populationInProgress = true;
                if (replace || !this.isFilled)
                {
                    this.isFilled = _load();
                }
                this.populationInProgress = false;
            }
            if (!recursive)
                this.getDbMetaTreeModel ().setStatusBarMessage ("Done");
        }
        if (recursive)
        {
            java.util.Enumeration e = this.children();
            while (e.hasMoreElements())
            {
                Object o = e.nextElement();
      

In [5]:
def get_training_corpus():
    dataset = raw_datasets["train"]
    for start_idx in range(0, len(dataset), 1000):
        samples = dataset[start_idx : start_idx + 1000]
        yield samples["whole_func_string"]

training_corpus = get_training_corpus()

In [6]:
from transformers import AutoTokenizer

old_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [7]:
example = '''static void myMethod() {
    System.out.println("I just got executed!");
  }

  public static void main(String[] args) {
    myMethod();
    myMethod();
    myMethod();
  }'''

tokens = old_tokenizer.tokenize(example)
tokens

['static',
 'void',
 'my',
 '##met',
 '##ho',
 '##d',
 '(',
 ')',
 '{',
 'system',
 '.',
 'out',
 '.',
 'print',
 '##ln',
 '(',
 '"',
 'i',
 'just',
 'got',
 'executed',
 '!',
 '"',
 ')',
 ';',
 '}',
 'public',
 'static',
 'void',
 'main',
 '(',
 'string',
 '[',
 ']',
 'ar',
 '##gs',
 ')',
 '{',
 'my',
 '##met',
 '##ho',
 '##d',
 '(',
 ')',
 ';',
 'my',
 '##met',
 '##ho',
 '##d',
 '(',
 ')',
 ';',
 'my',
 '##met',
 '##ho',
 '##d',
 '(',
 ')',
 ';',
 '}']

In [ ]:
tokenizer = old_tokenizer.train_new_from_iterator(training_corpus, 30000)